In [1]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'yellow-taxi-trips-2025' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

# I/ Market Demand & Seasonality


## Question 1: How does the demand for yellow taxis fluctuate over time (daily, weekly, monthly, and seasonally)?

In [11]:
# Query the BigQuery View
query_demand_over_time = """
SELECT *
FROM `yellow-taxi-trips-2025.views_fordashboard.demand_over_time`;
"""

# Run the query and store the result in a DataFrame
demand_over_time_df = client.query(query_demand_over_time).to_dataframe()
demand_over_time_df.head(10)

,trip_date,year,month,week,weekday,total_trips
0,2001-01-01,2001,1,0,2,11
1,2001-08-23,2001,8,33,5,1
2,2002-10-21,2002,10,42,2,36
3,2002-10-22,2002,10,42,3,53
4,2002-10-23,2002,10,42,4,53
5,2002-10-24,2002,10,42,5,51
6,2002-10-25,2002,10,42,6,53
7,2002-10-26,2002,10,42,7,36
8,2002-10-27,2002,10,43,1,46
9,2002-10-28,2002,10,43,2,25


In [12]:
type(demand_over_time_df)

pandas.core.frame.DataFrame

In [13]:
demand_over_time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   trip_date    1118 non-null   dbdate
 1   year         1118 non-null   Int64 
 2   month        1118 non-null   Int64 
 3   week         1118 non-null   Int64 
 4   weekday      1118 non-null   Int64 
 5   total_trips  1118 non-null   Int64 
dtypes: Int64(5), dbdate(1)
memory usage: 58.0 KB


In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [15]:
# Convert trip_date to datetime format
demand_over_time_df['trip_date'] = pd.to_datetime(demand_over_time_df['trip_date'])
demand_over_time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   trip_date    1118 non-null   datetime64[ns]
 1   year         1118 non-null   Int64         
 2   month        1118 non-null   Int64         
 3   week         1118 non-null   Int64         
 4   weekday      1118 non-null   Int64         
 5   total_trips  1118 non-null   Int64         
dtypes: Int64(5), datetime64[ns](1)
memory usage: 58.0 KB


In [16]:
demand_over_time_df.year.value_counts(normalize=True)

,proportion
year,
2024,0.32737
2022,0.326476
2023,0.326476
2002,0.00805
2025,0.004472
2001,0.001789
2003,0.000894
2008,0.000894
2009,0.000894


In [17]:
# Extract the current year
from datetime import datetime
current_year = datetime.now().year
current_year

2025

In [18]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_demand_over_time_df = demand_over_time_df[(demand_over_time_df['year'] >= 2020) & (demand_over_time_df['year'] <= current_year)]
filtered_demand_over_time_df.year.value_counts(normalize=True)

,proportion
year,
2024,0.332123
2022,0.331216
2023,0.331216
2025,0.004537
2021,0.000907


In [19]:
# 1. Daily Demand Trend
fig_daily = px.line(
    filtered_demand_over_time_df,
    x='trip_date',
    y='total_trips',
    title='Daily Taxi Demand Over Time',
    labels={'trip_date': 'Date', 'total_trips': 'Number of Trips'},
    template='plotly_dark'
)
fig_daily.show()

In [20]:
# 2. Weekly Demand Trend
weekly_demand = filtered_demand_over_time_df.groupby('week')['total_trips'].sum().reset_index()
fig_weekly = px.line(
    weekly_demand,
    x='week',
    y='total_trips',
    title='Weekly Taxi Demand Trend',
    labels={'week': 'Week Number', 'total_trips': 'Number of Trips'},
    template='plotly_dark'
)
fig_weekly.show()

In [21]:
# 3. Monthly Demand Trend
monthly_demand = filtered_demand_over_time_df.groupby('month')['total_trips'].sum().reset_index()
fig_monthly = px.bar(
    monthly_demand,
    x='month',
    y='total_trips',
    title='Monthly Taxi Demand Trend',
    labels={'month': 'Month', 'total_trips': 'Number of Trips'},
    template='plotly_dark'
)
fig_monthly.show()

In [22]:
filtered_demand_over_time_df['quarter'] = filtered_demand_over_time_df['trip_date'].dt.to_period("Q")
filtered_demand_over_time_df

<ipython-input-22-9d11e34f1b47>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,trip_date,year,month,week,weekday,total_trips,quarter
15,2021-12-31,2021,12,52,6,23,2021Q4
16,2022-01-01,2022,1,0,7,58857,2022Q1
17,2022-01-02,2022,1,1,1,54917,2022Q1
18,2022-01-03,2022,1,1,2,67990,2022Q1
19,2022-01-04,2022,1,1,3,70308,2022Q1
...,...,...,...,...,...,...,...
1112,2025-01-06,2025,1,1,2,1,2025Q1
1113,2025-01-16,2025,1,2,5,1,2025Q1
1114,2025-02-09,2025,2,6,1,1,2025Q1
1115,2025-03-02,2025,3,9,1,1,2025Q1


In [23]:
seasonal_demand = filtered_demand_over_time_df.groupby('quarter')['total_trips'].sum().reset_index()
seasonal_demand

,quarter,total_trips
0,2021Q4,23
1,2022Q1,8455467
2,2022Q2,9969918
3,2022Q3,8831679
4,2022Q4,9561129
5,2023Q1,8808178
6,2023Q2,9445156
7,2023Q3,7947309
8,2023Q4,9413471
9,2024Q1,8480258


In [24]:
# Convert 'quarter' column to string
seasonal_demand['quarter'] = seasonal_demand['quarter'].astype(str)

In [25]:
# 4. Seasonal Demand Trend (Grouping by Quarters)
fig_seasonal = px.line(
    seasonal_demand,
    x='quarter',
    y='total_trips',
    title='Seasonal Taxi Demand Trend',
    labels={'quarter': 'Quarter', 'total_trips': 'Number of Trips'},
    template='plotly_dark'
)
fig_seasonal.show()

## Question 2: What are the peak hours for yellow taxi trips in different boroughs and zones?

In [26]:
# Function to execute a BigQuery query and return a DataFrame

def query_to_dataframe(query: str) -> pd.DataFrame:
    """
    Executes a SQL query in BigQuery and returns a Pandas DataFrame.

    Parameters:
    - query (str): The SQL query to execute.

    Return:
    - pd.DataFrame : The DataFrame containing the results of the query.
    """
    try:
        df = client.query(query).to_dataframe()
        print(f"Query executed successfully. Retrieved {df.shape[0]} rows.")
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()


In [27]:
query_peak_hours_by_zone = """
SELECT *
FROM `yellow-taxi-trips-2025.views_fordashboard.peak_hours_by_zone`
"""
peak_hours_by_zone_df = query_to_dataframe(query_peak_hours_by_zone)
peak_hours_by_zone_df.head()

Query executed successfully. Retrieved 6123 rows.


,pickup_hour,Borough,Zone,total_trips
0,18,Manhattan,Midtown Center,423974
1,17,Manhattan,Midtown Center,418950
2,14,Manhattan,Upper East Side South,402753
3,15,Manhattan,Upper East Side South,402242
4,18,Manhattan,Upper East Side South,395383


In [28]:
peak_hours_by_zone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6123 entries, 0 to 6122
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pickup_hour  6123 non-null   Int64 
 1   Borough      6123 non-null   object
 2   Zone         6123 non-null   object
 3   total_trips  6123 non-null   Int64 
dtypes: Int64(2), object(2)
memory usage: 203.4+ KB


In [29]:
fig_borough = px.line(
    peak_hours_by_zone_df.groupby(["pickup_hour", "Borough"])["total_trips"].sum().reset_index(),
    x="pickup_hour",
    y="total_trips",
    color="Borough",
    title="Viewing peak hours by Borough",
    labels={"pickup_hour": "Heure", "total_trips": "Nombre de courses"},
    template="plotly_dark"
)

fig_borough.show()

In [30]:
# Pivot table pour avoir une matrice heures x Borough
pivot_df = peak_hours_by_zone_df.pivot_table(
    values="total_trips",
    index="pickup_hour",
    columns="Borough",
    aggfunc="sum"
)

pivot_df.head()

Borough,Bronx,Brooklyn,EWR,Manhattan,N/A,Queens,Staten Island,Unknown
pickup_hour,,,,,,,,
0,1546,23722,19,2544693,1060,368431,78,26116
1,1042,17059,11,1775442,858,147548,96,14399
2,787,12077,19,1206126,706,53890,64,9029
3,764,9646,25,783667,532,29282,59,5975
4,2826,11294,59,482595,478,31704,58,4171


In [31]:
# Remplacement des NaN par 0
#pivot_df = pivot_df.fillna(0)

# Heatmap of the busiest areas by hour
fig_zone = go.Figure(
    data=go.Heatmap(
        z=pivot_df.values,
        x=pivot_df.columns,
        y=pivot_df.index,
        colorscale="YlGnBu"
    )
)

fig_zone.update_layout(
    title="Heatmap des Heures de Pointe par Arrondissement",
    xaxis_title="Borough",
    yaxis_title="Hour",
    template="plotly_dark"
)

fig_zone.show()

In [32]:
# Aggregate trips per borough and hour
borough_hourly_demand = peak_hours_by_zone_df.groupby(["Borough", "pickup_hour"])["total_trips"].sum().reset_index()
borough_hourly_demand.head()

,Borough,pickup_hour,total_trips
0,Bronx,0,1546
1,Bronx,1,1042
2,Bronx,2,787
3,Bronx,3,764
4,Bronx,4,2826


In [33]:
# Create a facet bar chart
fig_facet = px.bar(
    borough_hourly_demand,
    x="pickup_hour",
    y="total_trips",
    color="Borough",
    facet_col="Borough",
    facet_col_wrap=3,  # Arrange in rows of 3
    title="Peak Hours for Yellow Taxi Trips by Borough",
    labels={"pickup_hour": "Hour of the Day", "total_trips": "Total Trips"},
    template="plotly_dark"
)

fig_facet.update_layout(
    showlegend=False,
    height=600,
    width=1200
)

fig_facet.show()

# II/ Customer Behavior & Ride Characteristics

## Question 4: What are the most popular pickup and drop-off locations, and how do they change over time?

In [34]:
query_popular_pickup_dropoff = """
SELECT *
FROM `yellow-taxi-trips-2025.views_fordashboard.popular_pickup_dropoff`
"""
popular_pickup_dropoff_df = query_to_dataframe(query_popular_pickup_dropoff)
popular_pickup_dropoff_df.head()

Query executed successfully. Retrieved 7254828 rows.


,trip_date,year,month,week,weekday,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone,total_trips
0,2023-12-31,2023,12,53,1,Manhattan,West Village,Manhattan,TriBeCa/Civic Center,69
1,2023-12-31,2023,12,53,1,Manhattan,SoHo,Manhattan,East Village,39
2,2023-12-31,2023,12,53,1,Manhattan,Clinton East,Manhattan,World Trade Center,11
3,2023-12-31,2023,12,53,1,Manhattan,Greenwich Village South,Queens,Long Island City/Hunters Point,3
4,2023-12-31,2023,12,53,1,Queens,East Elmhurst,Queens,Queensbridge/Ravenswood,1


In [35]:
popular_pickup_dropoff_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7254828 entries, 0 to 7254827
Data columns (total 10 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   trip_date        dbdate
 1   year             Int64 
 2   month            Int64 
 3   week             Int64 
 4   weekday          Int64 
 5   pickup_borough   object
 6   pickup_zone      object
 7   dropoff_borough  object
 8   dropoff_zone     object
 9   total_trips      Int64 
dtypes: Int64(5), dbdate(1), object(4)
memory usage: 588.1+ MB


In [36]:
popular_pickup_dropoff_df.year.value_counts(normalize=True)

,proportion
year,
2024,0.342217
2022,0.335579
2023,0.32213
2002,0.00005
2009,0.000009
2008,0.000008
2021,0.000003
2003,0.000002
2001,0.000002


In [37]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_popular_pickup_dropoff_df = popular_pickup_dropoff_df[(popular_pickup_dropoff_df['year'] >= 2020) & (popular_pickup_dropoff_df['year'] <= current_year)]

In [38]:
# Aggregate total trips by month
monthly_trips = (
    filtered_popular_pickup_dropoff_df.groupby(["year", "month"])["total_trips"]
    .sum()
    .reset_index()
)
monthly_trips.head()

,year,month,total_trips
0,2021,12,23
1,2022,1,2302210
2,2022,2,2772328
3,2022,3,3380929
4,2022,4,3354310


In [40]:

# Convert to datetime
monthly_trips["date"] = pd.to_datetime(
    monthly_trips["year"].astype(str) + "-" + monthly_trips["month"].astype(str)
)

# Plot
fig = px.line(
    monthly_trips,
    x="date",
    y="total_trips",
    markers=True,
    title="Monthly Trends in Taxi Demand",
    labels={"total_trips": "Total Trips", "date": "Date"},
)

fig.update_traces(line=dict(width=3))
fig.update_layout(template="plotly_dark", xaxis_tickangle=-45)
fig.show()


In [41]:
# Aggregate trips by pickup zone
top_pickups = (
    filtered_popular_pickup_dropoff_df.groupby("pickup_zone")["total_trips"]
    .sum()
    .nlargest(10)
    .reset_index()
)

top_pickups.head()

,pickup_zone,total_trips
0,JFK Airport,5562648
1,Upper East Side South,5236583
2,Midtown Center,4908741
3,Upper East Side North,4619443
4,Midtown East,3833007


In [43]:
# Plot
fig = px.bar(
    top_pickups,
    x="total_trips",
    y="pickup_zone",
    orientation="h",
    text="total_trips",
    title="Top 10 Pickup Locations",
    labels={"total_trips": "Total Trips", "pickup_zone": "Pickup Zone"},
)

fig.update_traces(marker_color="royalblue", texttemplate="%{text:,}", textposition="inside")
fig.update_layout(template="plotly_dark", xaxis_title="Total Trips")
fig.show()

In [44]:
# Aggregate total trips by borough and weekday
heatmap_data = (
    filtered_popular_pickup_dropoff_df.groupby(["pickup_borough", "weekday"])["total_trips"]
    .sum()
    .unstack()
)

heatmap_data.head()

weekday,1,2,3,4,5,6,7
pickup_borough,,,,,,,
Bronx,14319,21803,24506,25145,24368,24650,15567
Brooklyn,92280,85257,92576,96339,99123,103068,97686
EWR,986,612,428,523,741,737,584
Manhattan,11499995,11858448,14042387,14737920,15160357,14477255,14384145
N/A,4200,2898,2580,2694,3080,3216,3378


In [45]:

# Create heatmap
fig = go.Figure(
    data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale="viridis",
        colorbar=dict(title="Total Trips"),
    )
)

fig.update_layout(
    title="Taxi Demand Heatmap by Borough & Weekday",
    xaxis_title="Weekday",
    yaxis_title="Pickup Borough",
    template="plotly_dark",
)
fig.show()


## Question 5: What is the average trip distance, and how does it vary by borough, time of day, and season?

In [47]:
query_avg_trip_distance_analysis = """
SELECT *
FROM `yellow-taxi-trips-2025.views_fordashboard.avg_trip_distance_analysis`
"""
avg_trip_distance_analysis_df = query_to_dataframe(query_avg_trip_distance_analysis)
avg_trip_distance_analysis_df.head()

Query executed successfully. Retrieved 586207 rows.


,trip_date,year,month,season,pickup_hour,pickup_borough,dropoff_borough,avg_trip_distance
0,2001-01-01,2001,1,Winter,0,Queens,Unknown,18.630000
1,2001-01-01,2001,1,Winter,0,Queens,Manhattan,19.225000
2,2001-01-01,2001,1,Winter,0,Manhattan,Manhattan,3.323333
3,2001-01-01,2001,1,Winter,0,Manhattan,Brooklyn,10.230000
4,2001-01-01,2001,1,Winter,1,Manhattan,Manhattan,3.890000


In [48]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_avg_trip_distance_analysis_df = avg_trip_distance_analysis_df[(avg_trip_distance_analysis_df['year'] >= 2020) & (avg_trip_distance_analysis_df['year'] <= current_year)]
filtered_avg_trip_distance_analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 585927 entries, 278 to 586204
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   trip_date          585927 non-null  dbdate 
 1   year               585927 non-null  Int64  
 2   month              585927 non-null  Int64  
 3   season             585927 non-null  object 
 4   pickup_hour        585927 non-null  Int64  
 5   pickup_borough     585927 non-null  object 
 6   dropoff_borough    585927 non-null  object 
 7   avg_trip_distance  585927 non-null  float64
dtypes: Int64(3), dbdate(1), float64(1), object(3)
memory usage: 41.9+ MB


In [49]:
# Convert trip_date to datetime
filtered_avg_trip_distance_analysis_df["trip_date"] = pd.to_datetime(filtered_avg_trip_distance_analysis_df["trip_date"])

# Aggregate average trip distance by date
daily_avg_distance = (
    filtered_avg_trip_distance_analysis_df.groupby("trip_date")["avg_trip_distance"]
    .mean()
    .reset_index()
)

daily_avg_distance.head()

<ipython-input-49-09532ef2363a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,trip_date,avg_trip_distance
0,2021-12-31,9.505895
1,2022-01-01,10.026574
2,2022-01-02,11.315739
3,2022-01-03,11.142847
4,2022-01-04,10.818771


In [50]:
# Plot
fig = px.line(
    daily_avg_distance,
    x="trip_date",
    y="avg_trip_distance",
    title="Average Trip Distance Over Time",
    labels={"avg_trip_distance": "Avg Trip Distance (miles)", "trip_date": "Date"},
    markers=True,
)

fig.update_traces(line=dict(width=3))
fig.update_layout(template="plotly_dark", xaxis_tickangle=-45)
fig.show()


## Question 6: How many trips have only one passenger versus multiple passengers, and does this change seasonally?

In [52]:
query_passenger_trends_by_season = """
SELECT *
FROM `yellow-taxi-trips-2025.views_fordashboard.passenger_trends_by_season`
"""
passenger_trends_by_season_df = query_to_dataframe(query_passenger_trends_by_season)
passenger_trends_by_season_df.head()

Query executed successfully. Retrieved 2227 rows.


,trip_date,year,month,season,passenger_category,total_trips
0,2001-01-01,2001,1,Winter,Multiple Passengers,2
1,2001-01-01,2001,1,Winter,Single Passenger,9
2,2001-08-23,2001,8,Summer,Single Passenger,1
3,2002-10-21,2002,10,Fall,Single Passenger,31
4,2002-10-21,2002,10,Fall,Multiple Passengers,5


In [53]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_passenger_trends_by_season_df = passenger_trends_by_season_df[(passenger_trends_by_season_df['year'] >= 2020) & (passenger_trends_by_season_df['year'] <= current_year)]
filtered_passenger_trends_by_season_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2199 entries, 27 to 2225
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   trip_date           2199 non-null   dbdate
 1   year                2199 non-null   Int64 
 2   month               2199 non-null   Int64 
 3   season              2199 non-null   object
 4   passenger_category  2199 non-null   object
 5   total_trips         2199 non-null   Int64 
dtypes: Int64(3), dbdate(1), object(2)
memory usage: 126.7+ KB


In [54]:
# Aggregate total trips by passenger category
passenger_counts = filtered_passenger_trends_by_season_df.groupby("passenger_category")["total_trips"].sum().reset_index()
passenger_counts

,passenger_category,total_trips
0,Multiple Passengers,25310942
1,Single Passenger,82754168


In [55]:
# Create a donut chart
fig = px.pie(
    passenger_counts,
    names="passenger_category",
    values="total_trips",
    title="Proportion of Single vs. Multiple Passenger Trips",
    hole=0.4,  # This makes it a donut chart
    color="passenger_category",
    color_discrete_map={"Single Passenger": "red", "Multiple Passengers": "blue"}
)

fig.update_traces(textinfo="percent+label")

# Show the figure
fig.show()

In [57]:
# Aggregate data
treemap_data = filtered_passenger_trends_by_season_df.groupby(["season", "passenger_category"])["total_trips"].sum().reset_index()

# Define a seasonal color palette
season_colors = {
    "Winter": "blue",
    "Spring": "green",
    "Summer": "yellow",
    "Fall": "orange",
}

# Create the treemap
fig = px.treemap(
    treemap_data,
    path=["season", "passenger_category"],
    values="total_trips",
    color="season",
    color_discrete_map=season_colors,
    title="Passenger Distribution by Season (Treemap)",
)

fig.update_traces(textinfo="label+percent entry")

# Show the plot
fig.show()


In [58]:
# Convert trip_date to datetime
filtered_passenger_trends_by_season_df["trip_date"] = pd.to_datetime(filtered_passenger_trends_by_season_df["trip_date"])

<ipython-input-58-b108a65472d3>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [59]:
# Stacked Area Chart: Passenger Distribution Over Time

  # This helps visualize the proportion of single vs. multiple passengers over time.

fig = px.area(
    filtered_passenger_trends_by_season_df,
    x="trip_date",
    y="total_trips",
    color="passenger_category",
    title="Proportion of Single vs. Multiple Passenger Trips Over Time",
    labels={"total_trips": "Total Trips", "trip_date": "Date", "passenger_category": "Passenger Category"},
)

fig.update_layout(template="plotly_dark")
fig.show()
